In [1]:
import sys
sys.path.append('../')


In [2]:
from DPF import ShardedFilesDatasetConfig, DatasetReader

config = ShardedFilesDatasetConfig.from_path_and_columns(
    'example_video_dataset',
    video_name_col='video_name',
    text_col='caption'
)

reader = DatasetReader()
processor = reader.read_from_config(config)


/home/user/conda/envs/dpf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:00<00:00, 160.72it/s]


In [3]:
processor.df


,video_path,text,split_name
0,example_video_dataset/0/0.mp4,Businessman Rides In A Car,0
1,example_video_dataset/0/1.mp4,Workspace In Natural Light,0
2,example_video_dataset/0/2.mp4,Woman In Dress Walking In Tulip Field,0
3,example_video_dataset/0/3.mp4,Film Burns Overlay,0
4,example_video_dataset/0/4.mp4,Portrait Leader Of The Roman Army,0


## Create pipeline

In [6]:
from DPF.pipelines import FilterPipeline
from DPF.filters.videos.info_filter import VideoInfoFilter
from DPF.filters.videos.lita_filter import LITAFilter

pipeline = FilterPipeline("video_filter_example")
pipeline.add_datafilter(
    VideoInfoFilter,
    {'workers': 16},   
)
pipeline.add_datafilter(
    LITAFilter,
    {'batch_size': 2, 'workers': 1},
    devices=['cuda:0', 'cuda:1']
)


In [7]:
pipeline.run(processor)


[2024-05-22 00:31:58,930][INFO]: Starting filtering dataset example_video_dataset with video_filter_example pipeline
[2024-05-22 00:31:58,932][INFO]: Dataset path: example_video_dataset
[2024-05-22 00:31:58,933][INFO]: Dataset modalities: ['video', 'text']
[2024-05-22 00:31:58,933][INFO]: Dataset size: (5, 3)
[2024-05-22 00:31:58,934][INFO]: Dataset columns: Index(['video_path', 'text', 'split_name'], dtype='object')
[2024-05-22 00:31:58,935][INFO]: ----------------
[2024-05-22 00:31:58,935][INFO]: Starting stage 0: FilterPipelineStage(filter_class=<class 'DPF.filters.videos.info_filter.VideoInfoFilter'>, filter_kwargs={'workers': 16})


100%|██████████| 5/5 [00:00<00:00, 16.11it/s]

[2024-05-22 00:31:59,255][INFO]: Pipeline stage finished. New dataframe shape: (5, 9)
[2024-05-22 00:31:59,256][INFO]: ----------------
[2024-05-22 00:31:59,256][INFO]: Starting stage 1: FilterPipelineStage(filter_class=<class 'DPF.filters.multigpu_filter.MultiGPUDataFilter'>, filter_kwargs={'devices': ['cuda:0', 'cuda:1'], 'datafilter_class': <class 'DPF.filters.videos.lita_filter.LITAFilter'>, 'datafilter_params': {'batch_size': 2, 'workers': 1}})



You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/home/user/conda/envs/dpf/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:19<00:00,  6.40s/it]
Some weights of the model checkpoint at ./lita-vicuna-v1-3-13b-finetune were not used when initializing LitaLlamaForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.23.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.9.layer_norm1.weight', 'model.vision_tower.vi

[2024-05-22 00:32:23,568] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-05-22 00:32:23,708] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/home/user/conda/envs/dpf/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/user/conda/envs/dpf/lib/python3.11/site-packages/huggingface_hub/

[2024-05-22 00:33:01,542][INFO]: Pipeline stage finished. New dataframe shape: (5, 10)


In [9]:
processor.df


,video_path,text,split_name,is_correct,error,width,height,fps,duration,caption lita-vicuna-v1-3-13b-finetune prompt detailed_video
0,example_video_dataset/0/0.mp4,Businessman Rides In A Car,0,True,None,1280,720,25.000000,14.2800,The video is a single shot of a man wearing a ...
1,example_video_dataset/0/1.mp4,Workspace In Natural Light,0,True,None,1280,720,25.000000,10.0800,The video is a white room setup with a desk an...
2,example_video_dataset/0/2.mp4,Woman In Dress Walking In Tulip Field,0,True,None,1280,720,25.000000,12.1200,"The video is a short, artistic piece featuring..."
3,example_video_dataset/0/3.mp4,Film Burns Overlay,0,True,None,1280,720,23.976024,24.5245,"The video is a psychedelic, colorful, and trip..."
4,example_video_dataset/0/4.mp4,Portrait Leader Of The Roman Army,0,True,None,1280,720,29.970030,9.2092,The video features a man dressed in a metal su...


## Create video preprocessing pipeline

In [6]:
from DPF.pipelines import FilterPipeline
from DPF.filters.videos.info_filter import VideoInfoFilter
from DPF.filters.videos.farneback_filter import GunnarFarnebackFilter
from DPF.filters.videos.image_filter_adapter import ImageFilterAdapter
from DPF.filters.images.aesthetic_improved_filter import ImprovedAestheticFilter
from DPF.transforms import VideoFFMPEGTransforms, Resizer, ResizerModes

pipeline = FilterPipeline("video_preprocessing_filter_example")

# getting base info about videos
pipeline.add_datafilter(
    VideoInfoFilter,
    dict(workers=16),
    skip_if_columns_exist=True 
)

# resizing videos to 512 min size and transform to 24 fps
pipeline.add_transforms(
    VideoFFMPEGTransforms,
    dict(
        resizer=Resizer(ResizerModes.MIN_SIZE, 512),
        fps=24,
        preset="fast",
        crf=23,
        workers=8
    )
)

# calculate optical flow
pipeline.add_datafilter(
    GunnarFarnebackFilter,
    dict(pass_frames=12, min_frame_size=512, num_passes=12, workers=8),
)

# calculate aesthetic score on central frame
pipeline.add_datafilter(
    ImageFilterAdapter,
    dict(
        image_filter=ImprovedAestheticFilter('../weights/', device="cuda:0"),
        video_frame=0.5, # central frame
        workers=16
    )
)


In [7]:
!rm -rf example_video_dataset_test/
!cp -r example_video_dataset/ example_video_dataset_test/


In [8]:
from DPF import ShardedFilesDatasetConfig, DatasetReader

config = ShardedFilesDatasetConfig.from_path_and_columns(
    'example_video_dataset_test',
    video_name_col='video_name',
    text_col='caption'
)

reader = DatasetReader()
processor = reader.read_from_config(config)
processor.df


100%|██████████| 1/1 [00:00<00:00, 148.78it/s]


,video_path,text,split_name
0,example_video_dataset_test/0/0.mp4,Businessman Rides In A Car,0
1,example_video_dataset_test/0/1.mp4,Workspace In Natural Light,0
2,example_video_dataset_test/0/2.mp4,Woman In Dress Walking In Tulip Field,0
3,example_video_dataset_test/0/3.mp4,Film Burns Overlay,0
4,example_video_dataset_test/0/4.mp4,Portrait Leader Of The Roman Army,0


In [9]:
pipeline.run(processor)


[2024-05-22 00:48:58,056][INFO]: Starting filtering dataset example_video_dataset_test with video_preprocessing_filter_example pipeline
[2024-05-22 00:48:58,057][INFO]: Dataset path: example_video_dataset_test
[2024-05-22 00:48:58,058][INFO]: Dataset modalities: ['video', 'text']
[2024-05-22 00:48:58,058][INFO]: Dataset size: (5, 3)
[2024-05-22 00:48:58,059][INFO]: Dataset columns: Index(['video_path', 'text', 'split_name'], dtype='object')
[2024-05-22 00:48:58,060][INFO]: ----------------
[2024-05-22 00:48:58,060][INFO]: Starting stage 0: FilterPipelineStage(filter_class=<class 'DPF.filters.videos.info_filter.VideoInfoFilter'>, filter_kwargs={'workers': 16})


100%|██████████| 5/5 [00:00<00:00,  7.57it/s]

[2024-05-22 00:48:58,731][INFO]: Pipeline stage finished. New dataframe shape: (5, 9)
[2024-05-22 00:48:58,731][INFO]: ----------------
[2024-05-22 00:48:58,732][INFO]: Starting stage 1: TransformPipelineStage(transforms_class=<class 'DPF.transforms.video_ffmpeg_transforms.VideoFFMPEGTransforms'>, transforms_kwargs={'resizer': <DPF.transforms.resizer.Resizer object at 0x7fb866fe1850>, 'fps': 24, 'preset': 'fast', 'crf': 23, 'workers': 8})



100%|██████████| 5/5 [00:01<00:00,  3.07it/s]

[2024-05-22 00:49:00,419][INFO]: Pipeline stage finished. New dataframe shape: (5, 9)
[2024-05-22 00:49:00,420][INFO]: ----------------
[2024-05-22 00:49:00,421][INFO]: Starting stage 2: FilterPipelineStage(filter_class=<class 'DPF.filters.videos.farneback_filter.GunnarFarnebackFilter'>, filter_kwargs={'pass_frames': 12, 'min_frame_size': 512, 'num_passes': 12, 'workers': 8})



100%|██████████| 5/5 [00:03<00:00,  1.37it/s]

[2024-05-22 00:49:04,066][INFO]: Pipeline stage finished. New dataframe shape: (5, 10)
[2024-05-22 00:49:04,066][INFO]: ----------------
[2024-05-22 00:49:04,067][INFO]: Starting stage 3: FilterPipelineStage(filter_class=<class 'DPF.filters.videos.image_filter_adapter.ImageFilterAdapter'>, filter_kwargs={'image_filter': <DPF.filters.images.aesthetic_improved_filter.ImprovedAestheticFilter object at 0x7fb8619d7550>, 'video_frame': 0.5, 'workers': 16})



100%|██████████| 5/5 [00:01<00:00,  2.96it/s]

[2024-05-22 00:49:05,764][INFO]: Pipeline stage finished. New dataframe shape: (5, 11)


In [10]:
processor.df


,video_path,text,split_name,is_correct,error,duration,width,height,fps,optical_flow_farneback,improved_aesthetic_score_ViT-L/14
0,example_video_dataset_test/0/0.mp4,Businessman Rides In A Car,0,True,None,14.2800,910,512,24.000000,4.508,5.383147
1,example_video_dataset_test/0/1.mp4,Workspace In Natural Light,0,True,None,10.0800,910,512,24.000000,0.885,5.444565
2,example_video_dataset_test/0/2.mp4,Woman In Dress Walking In Tulip Field,0,True,None,12.1200,910,512,24.000000,8.294,5.549158
3,example_video_dataset_test/0/3.mp4,Film Burns Overlay,0,True,None,24.5245,910,512,23.976024,0.030,4.060539
4,example_video_dataset_test/0/4.mp4,Portrait Leader Of The Roman Army,0,True,None,9.2092,910,512,24.000000,3.984,5.920192
